# Chapter 2 : Getting started with MLflow for DL

## **Setting up MLflow**

### **Setting up MLflow to interact with a remote MLflow server**

In a corporate production environment, MLflow is usually hosted on a cloud server, which could be self-hosted or one of the Databricks' managed services in one of the cloud providers (such as AWS, Azure, or Google Cloud). In those cases, there is a requirement to set up your local development environment so that you can run your ML/DL experiment locally but interact with the MLflow server remotely. Next, we will describe how to do this using environment variables with the help of the following three steps:
1. In a bash shell command-line environment, define three new environment variables if you are using a Databricks-managed MLflow tracking server. The first environment variable is MLFLOW_TRACKING_URI, and the assigned value is databricks:
```bash
export MLFLOW_TRACKING_URI=databricks
export DATABRICKS_HOST=https://*******
export DATABRICKS_TOKEN=dapi******
```
2. The second environment variable is `DATABRICKS_HOST`. If your Databricks managed website looks like `https://dbc-*.cloud.databricks.com/`, then that's the value of the `DATABRICKS_HOST` variable (replace `*` with your actual website string).
3. The third environment variable is `DATABRICKS_TOKEN`. Navigate to your Databricks-managed website at `https://dbc-*.cloud.databricks.com/#setting/account`, click on Access Tokens, and then click on Generate New Token. You will see a pop-up window with a Comment field (which can be used to record why this token will be used) and expiration date

### **Implementing our first DL experiment with MLflow autologging**

Let's use the DL sentiment classifier we built previously.

In [2]:
# 1 First we need to import the MLflow module
import mlflow
# This will provide API for logging and loading models
# 2 Juste before we run the training code, we need to set up an active experiment using
# mlflow.set_experiment for the current running code
EXPERIMENT_NAME = "dl_model_chapter2"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 157653204083531205


This sets an experiment named `dl_model_chapter02` to be the current active experiment. If this experiment does not exist in your current tracking server, it will be created automatically.

#### **ENVIRONMENT VARIABLE**

*Note that you might need to set the tracking server URI using the MLFLOW_TRACKING_URI environment variable before you run your first experiment. If you are using a hosted Databricks server, implement the following*:
```bash
export MLFLOW_TRACKING_URI=databricks
```

*If you are using a local server, then set this environment variable to empty or the default localhost at port number 5000 as follows (note that this is our current section's scenario and assumes you are using a local server):*
```bash
export MLFLOW_TRACKING_URI= http://127.0.0.1:5000
```

In [3]:
# 3 Next, add one line of code to enable autologging in MLflow:
mlflow.pytorch.autolog()

This will allow the default parameters, metrics, and model to be automatically logged to the MLflow tracking server.

### **AUTOLOGGING IN MLFLOW**

*Autologging in MLflow is still in experiment mode (as of version 1.20.2) and might change in the future. Here, we use it to explore the MLflow components since it only requires one line of code to automatically log everything of interest. In the upcoming chapters, we will learn about and implement additional ways to perform tracking and logging in MLflow. Also, note that currently, autologging in MLflow for PyTorch (as of version 1.20.2) only works for the PyTorch Lightning framework, not any arbitrary PyTorch code.*

In [4]:
import torch
import flash
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")

datamodule = TextClassificationData.from_csv(
    "review",
    "sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    batch_size=4,
)

classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes)
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())
trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")

/home/ibrahim/miniconda3/envs/mlflow/lib/python3.10/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and may be removed in the future. Please access them via the appropriate Weights Enum instead.
  warnings.warn(


./data/imdb.zip:   0%|          | 0/15575 [00:00<?, ?KB/s]

  0%|          | 0/22500 [00:00<?, ?ex/s]

  0%|          | 0/2500 [00:00<?, ?ex/s]

  0%|          | 0/2500 [00:00<?, ?ex/s]

...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

2022/12/25 18:20:02 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpgrl1jd2b/model/data, flavor: pytorch), fall back to return ['torch==1.13.1', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


In [5]:
# 4 Use the Python context manager with statement to start the experiment run by calling mlflow.start_run:
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter02"):
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 5624it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6503999829292297, 'test_cross_entropy': 0.6261751651763916}
--------------------------------------------------------------------------------


We can put all the code into a file named `first_dl_with_mlflow.py` and run the command:
```bash
python first_dl_with_mlflow.py
```

6. Now, we can open the MLflow UI locally to see what has been logged in the local tracking server by navigating to `http://127.0.0.1:5000/`. Here, you will see that a new experiment (`dl_model_chapter02`) with a new run (`Run Name = chapter02`) has been logged:

## **Exploring MLflow's components and usage patterns**

### **Exploring experiments and runs in MLflow**

**Experiment** is a **first-class entity in the MLflow APIs**. **This makes sense as data scientists and ML engineers need to run lots of experiments** in order to build a working model that meets the requirements. However, **the idea of an experiment goes beyond just the model development stage and extends to the entire life cycle of the ML/DL development and deployment**. So, this means that when we do retraining or training for a production version of the model, we need to treat them as production-quality experiments. This unified view of experiments builds a bridge between the offline and online production environments. Each experiment consists of many runs where you can either **change the model parameters**, **input data**, or even **model type** for each run. So, an experiment is an umbrella entity containing a series of runs.

Now, let's explore the MLflow experiments in a hands-on way. Run the following MLflow command line to interact with the tracking server:
```bash
mlflow experiments search
```
If your `MLFLOW_TRACKING_URI` environment variable points to a remote tracking server, then it will list all the experiments that you have read access to. The command lists the three columns of the experiment property: Experiment Id (an integer), Name (a text field that can be used to describe the experiment name), and Artifact Location (by default, this is located in the mlruns folder underneath the directory where you execute the MLflow commands). The mlruns folder is used by a filesystem-based MLflow tracking server to store all the metadata of experiment runs and artifacts.

### **THE COMMAND-LINE INTERFACE (CLI) VERSUS REST APIS VERSUS PROGRAMMING LANGUAGE-SPECIFIC APIS**

MLflow provides three different types of tools and APIs to interact with the tracking server. Here, the CLI is used so that we can explore the MLflow components.

So, let's explore a specific MLflow experiment, as follows:

1. First, create a new experiment using the MLflow CLI, as follows:
```bash
mlflow experiments create -n dl_model_chapter02
```
The preceding command creates a new experiment named `dl_model_chapter02`. If you have already run the first DL model with MLflow autologging in the previous section, the preceding command will cause an error message
2. Now, let's examine the relationship between experiments and runs. If you look carefully at the URL of the run page you will see something similar to the following:
`http://127.0.0.1:5000/#/experiments/1/runs/2f2ec6e72a5042f891abe0d3a533eec7` As you might have gathered, the integer after the experiments path is the experiment ID. Then, after the experiment ID, there is a runs path, followed by a GUID-like random string, which is the run ID. So, now we understand how the runs are organized under the experiment with a globally unique ID (called a run ID).

Knowing a run's globally unique ID is very useful. This is because we can retrieve the run's logged data using `run_id`. If you use the mlflow runs describe `--run-id <run_id>` command line, you can get the list of metadata that this run has logged. For the experiment we just ran, the following shows the full command with the run ID:
```bash
mlflow runs describe –-run-id 9b1034c2b4c54125ae78705b62514b1c
```

This will present all the metadata about this run in JSON format. This metadata includes parameters used by the model training; metrics for measuring the accuracy of the model in training, validation, and testing; and more. The same data is also presented in the MLflow UI. Note that the powerful MLflow CLI will allow very convenient exploration of the MLflow logged metadata and artifacts as well as enabling shell script-based automation, as we will explore in the upcoming chapters.

## **Exploring MLflow models and their usages**

### **THE MLFLOW TRACKING SERVER'S BACKEND STORE AND ARTIFACT STORE**

An MLflow tracking server has two types of storage: first, a backend store, which stores experiments and runs metadata along with params, metrics, and tags for runs; and second, an artifact store, which stores larger files such as serialized models, text files, or even generated plots for visualizing model results. For the purpose of simplicity, in this chapter, we are using a local filesystem for both the backend store and the artifact store. However, some of the more advanced features such as model registry are not available in a filesystem-based artifact store. In later chapters, we will learn how to use a model registry.

Let's look at the list of artifacts, one by one:
* `model_summary.txt`: At the root folder level, this file looks similar to the following output if you click on it. It describes the model metrics and the layers of the DL model and provides a quick overview of what the DL model looks like in terms of the number and type of neural network layers, the number and size of the parameters, and the type of metrics used in training and validation. This is very helpful when the DL model architecture is needed to be shared and communicated among team members or stakeholders.
* The `model` folder: This folder contains a subfolder, called data, and three files called `MLmodel`, `conda.yaml`, and `requirements.txt`:
  * `MLmodel`: This file describes the flavor of the model that MLflow supports. Flavor is MLflow-specific terminology. It describes how the model is saved, serialized, and loaded. For our first DL model, the following information is stored in an `MLmodel` file
  * `conda.yaml`: This is a conda environment definition file used by the model to describe our dependencies
  * `requirements.txt`: This is a Python pip-specific dependency definition file. It is just like the pip section in the conda.yaml file, as shown in Figure 2.16.
data: This is a folder that contains the actual serialized model, called model.pth, and a description file, called pickle_module_info.txt, whose content for our first DL experiment is as follows: `mlflow.pytorch.pickle_module`

This means the model is serialized using a PyTorch-compatible pickle serialization method provided by MLflow. This allows MLflow to load the model back to memory later if needed

### **MODEL REGISTRY VERSUS MODEL LOGGING**

The MLflow model registry requires a relational database such as MySQL as the artifact store, not just a plain filesystem. Therefore, in this chapter, we will not explore it yet. Note that a model registry is different from model logging in that, for each run, you want to log model metadata and artifacts. However, only for certain runs that meet your production requirements, you may want to register them in the model registry for production deployment and version control. In later chapters, we will learn how to register models.

By now, you should have a good understanding of the list of files and metadata about the model and the serialized model (along with the .pth file extension in our experiment, which refers to a PyTorch serialized model) logged in the MLflow artifact store.

## **Exploring MLflow code tracking and its usages**

When exploring the metadata of the run, we can also discover how the code is being tracked. As shown in the MLflow UI and the command-line output in JSON, the code is tracked in three ways: a filename, a Git commit hash, and a source type. You can execute the following command line:

```bash
mlflow runs describe --run-id 9b1034c2b4c54125ae78705b62514b1c | grep mlflow.source
```
Based on this 9b1034c2b4c54125ae78705b62514b1c Git commit hash, we can go on to find the exact copy of the Python code we used.

Note that, here, the source type is `LOCAL`. This means that we execute the MLflow-enabled source code from a local copy of the code.

### **LOCAL VERSUS REMOTE GITHUB CODE**

If the source is a local copy of the code, there is a caveat regarding the Git commit hash that you see in the MLflow tracking server. If you make code changes locally but forget to commit them and then immediately start an MLflow experiment tracking run, MLflow will only log the most recent Git commit hash. We can solve this problem in one of two ways:

1. Commit our code changes before running the MLflow experiment
2. Use remote GitHub code to run the experiment.

Since the first method is not easily enforceable, the second method is preferred. Using remote GitHub code to run a DL experiment is an advanced topic that we will explore in later chapters.

So far, we have learned about the MLflow tracking server, experiments, and runs. Additionally, we have logged metadata about runs such as parameters and metrics, examined code tracking, and explored model logging. These tracking and logging capabilities ensure that we have a solid ML experiment management system, not only for model development but also for model deployment in the future, as we need to track which runs produce the model for production. Reproducibility and provenance-tracking are the hallmarks of what MLflow provides. In addition to this, MLflow provides other components such as MLproject for standardized ML project code organization, a model registry for model versioning control, model deployment capabilities, and model explainability tools.